In [99]:
pip install sentence_transformers

     ---------------------------------------- 0.0/138.0 kB ? eta -:--:--
     ---------- -------------------------- 41.0/138.0 kB 991.0 kB/s eta 0:00:01
     -------------------------------------- 138.0/138.0 kB 2.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/171.5 kB ? eta -:--:--
   ---------------------------------------- 171.5/171.5 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.0 MB ? eta -:--:--
   - -------------------------------------- 0.4/9.0 MB 13.5 MB/s eta 0:00:01
   ---- ----------------------------------- 1.0/9.0 MB 12.3 MB/s eta 0:00:01
   ------- -------------------------------- 1.7/9.0 MB 13.8 MB/s eta 0:00:01
   ------------ --------------------------- 2.8/9.0 MB 16.3 MB/s eta 0:00:01
   ------------------ --------------------- 4.3/9.0 MB 19.5 MB/s eta 0:00:01
   --------------------------- ------------ 6.1/9.0 MB 23.1 MB/s eta 0:00:01
   --------------------------- ------------ 6.3/9.0 MB 23.7 MB/s eta 0:00:01
   -----

In [130]:
# import librosa
# import librosa.display
# import IPython.display as ipd
# from IPython.display import Audio

# import IPython.display as ipd

import numpy as np
import pandas as pd
# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?

# from tqdm import tqdm
# import np_utils
# import matplotlib.pyplot as plt
# import seaborn as sns

# import winsound as sd
# import glob
# import os
# import json
# import shutil
# import sys
# import logging
# import unicodedata
# from shutil import copyfile
# import warnings
# if not sys.warnoptions:
#     warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", category=DeprecationWarning) 


# # import json, pandas as pd # ?? 이러면 어떻게 되는 거지?



# import pydub #오디오 파일을 다루는 라이브러리 : 변환 조작 재생 분석 등
# import gTTS # 텍스트를 음성으로 변환하는 라이브러리. 나는 당장 쓸 일 없을 듯?
# import speech_recognition as sr #음성인식 라이브러리 - 텍스트로 변환
# import pyaudio

from sklearn.preprocessing import StandardScaler, OneHotEncoder, scale, MinMaxScaler
# from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 


import keras
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint,EarlyStopping
from keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical

# import tensorflow
from sentence_transformers import SentenceTransformer


# from transformers import PreTrainedTokenizer

In [2]:
# 오류 및 학습 완료 확인 beef 코드 
def beepError():
    fr = 1000    # range : 37 ~ 32767
    du = 2000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
def beepSuccess():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)

In [9]:
# 기본적인 사전 설정
rlrp = ReduceLROnPlateau(monitor='loss', factor=0.4, verbose=0, patience=5, min_lr=0.0000001) #learning rate 조절 
modelpath = './model/model_{epoch:02d}-{val_accuracy:.4f}.keras'
mcp = ModelCheckpoint(
    modelpath,     #저장할 모델의 경로
  monitor = 'val_accuracy', #val_acc를 기준으로 전보다 모델이 나아지는 걸 확인
  save_best_only = True,    #나아진 결과만 저장
#     save_weights_only=True , #이걸 써 줘야 weights.h5로 저장 가능하다.
  verbose = 1               #과정을 출력
)

#전보다 나아지지 않으면 학습중단

es = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 5      # 전보다 나아지지 않아도 실행할 횟수
)

In [125]:
#데이터 불러오기
X = np.load('./features5.npy') 
train = pd.read_csv('e:/Data2/csv/train.csv')

In [52]:
display(y)
print(X.shape)

,Angry,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,0,0,0,0,1,0,0
1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0
3,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
29314,0,0,0,0,1,0,0
29315,0,0,0,0,1,0,0
29316,0,0,0,0,1,0,0
29317,0,0,0,0,1,0,0


(29319, 58)


In [102]:
# 임베딩 함수 설정
class text_embedding():
    def __init__(self, model_name):
        self.model_name = model_name

    def fit(self, X, y=None):
        return self
  
    def transform(self, X):
        embedding_model = SentenceTransformer(self.model_name)
        embedding_vec = embedding_model.encode(train['발화문'])
        X_val = np.concatenate((X, embedding_vec), axis = 1)
        return X_val

## CNN 모델(입력 형태 : 3차원)

In [5]:
# from keras.models import Sequential
# from tensorflow.keras.layers import InputLayer, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
def first_model(x_train):
    model=Sequential()
    model.add(InputLayer(shape=(200,44)))
    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(256, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Conv1D(128, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))
    model.add(Dropout(0.2))

    model.add(Conv1D(64, kernel_size=5, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=5, strides = 2, padding = 'same'))

    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.3))

    model.add(Dense(units=7, activation='softmax'))
    model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

    #model.summary()
    return model

In [112]:
scaler = StandardScaler()
encoder = OneHotEncoder()

pre_trained_models = ['sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
'sentence-transformers/multi-qa-distilbert-cos-v1',
'jhgan/ko-sroberta-multitask',
'all-distilroberta-v1',
'jhgan/ko-sbert-multitask',
'all-MiniLM-L12-v2', 'jhgan/ko-sroberta-sts']


y =train.iloc[:,-7:]
for i in pre_trained_models:
    txt_embed = text_embedding(model_name = i)
    X = txt_embed.transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, shuffle=True)
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    X_train_scaled = np.expand_dims(X_train_scaled, axis=2)
    X_test_scaled = np.expand_dims(X_test_scaled, axis=2)
    model = first_model(X_train)
    history=model.fit(X_train_scaled, y_train, batch_size=64, epochs=50, validation_data=(X_test_scaled, y_test), callbacks=[rlrp])

    test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
    print("Pre-trained Model: ", i)
    print("Test Accuracy: ",test_acc)

C:\Users\smhrd\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch 1/50


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv1d" is incompatible with the layer: expected axis -1 of input shape to have value 44, but received input with shape (None, 3898, 1)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 3898, 1), dtype=float32)
  • training=True
  • mask=None

## SVM(Support Vector Model)

In [84]:
pre_trained_models = ['sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens',
'sentence-transformers/multi-qa-distilbert-cos-v1',
'jhgan/ko-sroberta-multitask',
'all-distilroberta-v1',
'jhgan/ko-sbert-multitask',
'all-MiniLM-L12-v2', 'jhgan/ko-sroberta-sts']


accuracy_history = []
y = train['감정']

#K-fold cross validation (train data 800개, test data 200개)

# kf = KFold(n_splits=5, shuffle=True, random_state=50) 
# #그룹을 다섯개로 나눔 (1000개의 데이터를 5개그룹으로 나누어 한개의 그룹이 test용 데이터가 됨)

# for train_index, test_index in kf.split(X):
#     #데이터 스플릿(5개의 그룹으로 나누었으니 train용과 test용을 구분함)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
    

X_train, X_test, y_train,y_test =train_test_split(X,y,random_state=5,stratify=y,test_size=0.3)
#MinMAx Scaler을 이용한 특징 벡터 전처리
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#분류기 커널 설정
clf = SVC(C=100, kernel='rbf', probability=True)
#분류기 학습
clf.fit(X_train_scaled, y_train)
#예측 결과
y_pred = clf.predict(X_test_scaled)
accuracy_history.append(accuracy_score(y_pred, y_test))
    

    
print("정확도 :", np.mean(accuracy_history))

각 분할의 정확도 기록:  [0.5160300136425648]
평균 정확도 : 0.5160300136425648


In [91]:
y_pred.shape

(8796,)

## ANN (Artificial Neural Network)

In [48]:
np.unique(y_train)

array([0, 1], dtype=int64)

In [128]:
y =train.iloc[:,-7:]

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=5,stratify=y,test_size=0.3)

# scaler = MinMaxScaler()
scaler =StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Define the model architecture
model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=7, activation='softmax'))  # Adjusted to use np.unique for flexibility

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, verbose=1, validation_data=(X_test_scaled, y_test),callbacks=[rlrp,mcp,es])

# Retrieve training and validation accuracy
train_accuracy = history.history['accuracy'][-1]  # Last epoch accuracy
val_accuracy = history.history['val_accuracy'][-1]  # Last epoch validation accuracy

# Predict on test set
y_pred = model.predict(X_test_scaled)
# 임계값 설정
threshold = 0.3

# 확률이 임계값보다 크면 1로, 작으면 0으로 분류
y_pred_labels = (y_pred > threshold).astype(int)
# y_pred_labels = np.argmax(y_pred, axis=1)

# Calculate test accuracy
test_accuracy = accuracy_score(y_test, y_pred_labels)

# Print accuracies
print('Training accuracy:', train_accuracy)
print('Validation accuracy:', val_accuracy)
print('Test accuracy:', test_accuracy)

Epoch 1/100


C:\Users\smhrd\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


614/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3898 - loss: 1.6340
Epoch 1: val_accuracy did not improve from 0.78013
642/642 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.3909 - loss: 1.6312 - val_accuracy: 0.4549 - val_loss: 1.4905 - learning_rate: 0.0010
Epoch 2/100
612/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4419 - loss: 1.5061
Epoch 2: val_accuracy did not improve from 0.78013
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4423 - loss: 1.5050 - val_accuracy: 0.4681 - val_loss: 1.4512 - learning_rate: 0.0010
Epoch 3/100
619/642 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4617 - loss: 1.4526
Epoch 3: val_accuracy did not improve from 0.78013
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4617 - loss: 1.4526 - val_accuracy: 0.4748 - val_loss: 1.4234 - learning_rate: 0.0010
Epoch 4/100
642/642 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4709 - loss: 1.4326
Epoch 4: val_accuracy did not improve from 0.78013
642/642 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/s

In [83]:
# emotion_list = ['Angry','Disgust','Fear','Happiness','Neutral','Sadness','Surprise']

# #softmax 함수를 사용하여 각 클래스에 속할 확률 계산
# probabilities = np.exp(y_pred) / np.sum(np.exp(y_pred), axis=1, keepdims=True)
print(y_pred[0])
print(probabilities[0])
print(sum(y_pred[0]))
print(sum(probabilities[0]))
# # 각 클래스에 속할 확률 출력
# for i, prob in enumerate(probabilities):
#     print(f"""당신의 {i + 1}번째 감정은 {emotion_list[0]}일 확률 : {prob[0]},{emotion_list[1]}일 확률 : {prob[1]},{emotion_list[2]}일 확률 : {prob[2]},{emotion_list[3]}일 확률 : {prob[3]},{emotion_list[4]}일 확률 : {prob[4]},{emotion_list[5]}일 확률 : {prob[5]},
#     {emotion_list[6]}일 확률 : {prob[0]}입니다.""")

[0.24236138 0.13584226 0.04317892 0.3490404  0.14114809 0.03996916
 0.04845984]
[0.15685809 0.14100881 0.12852958 0.17451671 0.14175898 0.1281177
 0.12921013]
1.0000000521540642
0.9999999850988388


## Xgboost

In [57]:
pip install xgboost

   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 640.0 kB/s eta 0:02:36
   ---------------------------------------- 0.1/99.8 MB 2.1 MB/s eta 0:00:48
   ---------------------------------------- 0.5/99.8 MB 4.8 MB/s eta 0:00:21
   ---------------------------------------- 1.0/99.8 MB 6.5 MB/s eta 0:00:16
    --------------------------------------- 1.8/99.8 MB 8.6 MB/s eta 0:00:12
   - -------------------------------------- 2.6/99.8 MB 10.5 MB/s eta 0:00:10
   - -------------------------------------- 4.0/99.8 MB 13.4 MB/s eta 0:00:08
   -- ------------------------------------- 5.7/99.8 MB 16.5 MB/s eta 0:00:06
   --- ------------------------------------ 8.1/99.8 MB 20.7 MB/s eta 0:00:05
   ---- ----------------------------------- 11.1/99.8 MB 34.4 MB/s eta 0:00:03
   ------ --------------------------------- 15.3/99.8 MB 65.6 MB/s eta 0:00:02
   -------- ------------------------------- 20.1/99.8 MB 93.9 MB/s eta 0:

In [60]:
from xgboost import XGBClassifier #xgboost 모델이 좋다길래 ..!!
from sklearn.metrics import accuracy_score

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05) #1000개의 가지? epoch? , 0.05 학습률
xgb.fit(X_train_scaled, y_train) #학습

y_preds = xgb.predict(X_test_scaled) #검증

print('Accuracy: %.2f' % accuracy_score(y_test,y_preds))

Accuracy: 0.29


In [131]:
model = keras.models.load_model('./model/model_06-0.8196.keras')